From modin main and snowpark python repo: `pip install -e .`

In [1]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVLRctowEPwVj%2FqMJQxJQANkKCSDO2nwYEjavAlbBoEtKTo5hn59ZQOd5CGZvmlOu7d7tze4PRS598YNCCWHqO0T5HGZqFTIzRCtlvetHvLAMpmyXEk%2BREcO6HY0AFbkmo5Lu5UL%2FlpysJ5rJIHWH0NUGkkVAwFUsoIDtQmNxz8faOATygC4sU4OnSkpCKe1tVZTjKuq8quOr8wGB4QQTPrYoWrIN%2FROQn%2BtoY2yKlH5hXJwM30i0cakW0s4hFOIzsTvQp5W8JXK%2BgQCOlsuo1Y0j5fIG1%2BmmygJZcFNzM2bSPhq8XAyAM5B%2FDh%2Fns1X8Z0PUlVZzvY8UYUurevmuxfOeIpztRFuR%2BF0iPRepCJbpbpzt11c98PS3oTymu%2FiLCnXu13KfnVfe7OJMZuoM9W%2FE%2BQ9XRIN6kRDgJKHss7RuhIJrlqk2yKdJenQK0K7Pb8X9F%2BQN3U5Cslsw7yYrS2uxR9f7S1rzDGt8T%2FfmB%2F2vSqDzBx%2FhLIbPr8E%2BgYDKFzHhE6XQhsDZvS%2F8w%2Fwe9b52B7d%2FsNppHKRHL17ZQpmP4%2Bn7bebikhbWQOlvGAiH6ep4QAupjxX1cRwZt1NW1NyhEcn1Y9XPfoL&RelayState=ve

# Example 1: Start with small dataset perform some processing, then join with large data

In [49]:
# emphemeral dataframe/lookup table 
# List of U.S. federal holidays
us_holidays = [
    ("New Year's Day", "2025-01-01"),
    ("Martin Luther King Jr. Day", "2025-01-20"),
    ("Presidents' Day", "2025-02-17"),
    ("Memorial Day", "2025-05-26"),
    ("Juneteenth National Independence Day", "2025-06-19"),
    ("Independence Day", "2025-07-04"),
    ("Labor Day", "2025-09-01"),
    ("Columbus Day", "2025-10-13"),
    ("Veterans Day", "2025-11-11"),
    ("Thanksgiving Day", "2025-11-27"),
    ("Christmas Day", "2025-12-25")
]

# Create DataFrame
df_us_holidays = pd.DataFrame(us_holidays, columns=["Holiday", "Date"])

# Convert Date column to datetime
df_us_holidays["Date"] = pd.to_datetime(df_us_holidays["Date"])

In [50]:
assert df_us_holidays.get_backend() == 'Pandas'  # with auto, we should expect this to be local

In [51]:
# Add new columns for transformations
df_us_holidays["Day_of_Week"] = df_us_holidays["Date"].dt.day_name()
df_us_holidays["Month"] = df_us_holidays["Date"].dt.month_name()

In [52]:
df_us_holidays

,Holiday,Date,Day_of_Week,Month
0,New Year's Day,2025-01-01,Wednesday,January
1,Martin Luther King Jr. Day,2025-01-20,Monday,January
2,Presidents' Day,2025-02-17,Monday,February
3,Memorial Day,2025-05-26,Monday,May
4,Juneteenth National Independence Day,2025-06-19,Thursday,June
5,Independence Day,2025-07-04,Friday,July
6,Labor Day,2025-09-01,Monday,September
7,Columbus Day,2025-10-13,Monday,October
8,Veterans Day,2025-11-11,Tuesday,November
9,Thanksgiving Day,2025-11-27,Thursday,November


In [53]:
%%time
#Note that without auto-switching, this took 2.5 min
for index, row in df_us_holidays.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 124 ms, sys: 5.68 ms, total: 130 ms
Wall time: 127 ms


In [54]:
pandas_df = df_us_holidays #.move_to("pandas") # remove this once we have auto-switching

In [55]:
assert pandas_df.get_backend() == 'Pandas' # With auto we expect this to be local

In [56]:
%%time
for index, row in pandas_df.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 117 ms, sys: 3.5 ms, total: 121 ms
Wall time: 119 ms


### Another mini example: generate synthetic data

In [58]:
%%time
# Generate 10 million transactions (This is also very slow with Snowpark pandas)
import uuid
import pandas
dates = pandas.date_range(start="2025-01-01", end="2025-12-31", freq="D")
num_transactions = 10000000
data = {
    "Transaction_ID": [str(uuid.uuid4()) for _ in range(num_transactions)],
    "Date": np.random.choice(dates, num_transactions),
    "Revenue": np.random.uniform(10, 1000, num_transactions)
}
df_transactions = pd.DataFrame(data)

CPU times: user 18.1 s, sys: 8.98 s, total: 27.1 s
Wall time: 27.1 s


In [59]:
assert df_transactions.get_backend() == "Pandas"

### 💡 Automatic switching speeds up loops/iterations on small data + inline creation of dataframes

## Example 2: Demonstrate that when data is prefiltered via SQL, the engine choice changes

In [60]:
# Run the following to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
session.sql('''
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);''').collect()
session.sql('''SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));''').collect()
session.sql('''INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
''').collect()

[Row(number of rows inserted=10000000)]

In [61]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

In [62]:
len(df_transactions)

10000000

In [63]:
# df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [64]:
df_transactions.groupby("DATE").sum()["REVENUE"]

DATE
2024-01-01    10967281.0
2024-01-02    10987086.0
2024-01-03    11034799.0
2024-01-04    11108325.0
2024-01-05    11055200.0
                 ...    
2025-03-30    11008169.0
2025-03-31    10985845.0
2025-04-01    10996598.0
2025-04-02    10938827.0
2025-04-03    11104779.0
Name: REVENUE, Length: 459, dtype: float64

In [65]:
assert df_transactions.get_backend() == "Snowflake"

In [66]:
# Filter to records in last 7 days
df_transactions_filter = pd.read_snowflake("SELECT * FROM revenue_transactions WHERE Date >= DATEADD( 'days', -7, current_date ) and Date < current_date")

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [67]:
len(df_transactions_filter)

152288

In [68]:
assert df_transactions_filter.get_backend() == "Pandas" # This should work once auto switching is on

In [69]:
df_transactions_filter.groupby("DATE").sum()["REVENUE"]

DATE
2025-03-27    10973986.0
2025-03-28    11096020.0
2025-03-29    10885425.0
2025-03-30    11008169.0
2025-03-31    10985845.0
2025-04-01    10996598.0
2025-04-02    10938827.0
Name: REVENUE, dtype: float64

### 💡 Automatic switching means that pandas work well for both small and large data

# Example 3: 

Forecast using last year's transaction data via a custom apply function

In [70]:
start_date = pd.Timestamp("2025-10-01")
end_date = pd.Timestamp("2025-10-31")

df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]

In [71]:
len(df_transactions_filtered)

653550

In [72]:
df_transactions_filtered_pandas = df_transactions #.move_to("pandas")

In [73]:
# Forecasting function using df.apply

def forecast_revenue(df, start_date, end_date):
    # Filter data from last year
    df_filtered = df[(df["DATE"] >= start_date - pd.Timedelta(days=365)) & (df["DATE"] < start_date)]
    
    # Append future dates to daily_avg for prediction
    future_dates = pd.date_range(start=start_date, end=end_date, freq="D")
    df_future = pd.DataFrame({"DATE": future_dates})

    # Group by DATE and calculate the mean revenue
    daily_avg = df_filtered.groupby("DATE")["REVENUE"].mean().reset_index()
    daily_avg["DATE"] = daily_avg["DATE"].astype('datetime64[ns]')
    # Merge future dates with predicted revenue, filling missing values
    df_forecast = df_future.merge(daily_avg, on="DATE", how="left")
    
    import numpy as np
    # Fill missing predicted revenue with overall mean from last year
    df_forecast["PREDICTED_REVENUE"] = np.nan
    df_forecast["PREDICTED_REVENUE"].fillna(daily_avg["REVENUE"].mean(), inplace=True)
    df_forecast["PREDICTED_REVENUE"] = df_forecast["PREDICTED_REVENUE"].astype("float")
    return df_forecast

In [74]:
# Example usage
df_forecast = forecast_revenue(df_transactions, start_date, end_date)
df_forecast

Transferring data from Pandas to Snowflake ...:   0%|          | 0/2 [00:00<?, ?it/s]

,DATE,REVENUE,PREDICTED_REVENUE
0,2025-10-01,NaN,505.173054
1,2025-10-02,NaN,505.173054
2,2025-10-03,NaN,505.173054
3,2025-10-04,NaN,505.173054
4,2025-10-05,NaN,505.173054
5,2025-10-06,NaN,505.173054
6,2025-10-07,NaN,505.173054
7,2025-10-08,NaN,505.173054
8,2025-10-09,NaN,505.173054
9,2025-10-10,NaN,505.173054


In [75]:
# df_forecast["DATE"] = df_forecast["DATE"].dt.strftime('%Y-%m-%d')

In [76]:
def adjust_for_holiday_weekend(row):
    # For national holidays, revenue down 20% since stores are closed. For weekends, revenue is up 20% due to increased activity.
    if row["DATE"].strftime('%Y-%m-%d') in list(df_us_holidays["Date"].dt.strftime('%Y-%m-%d')): 
        return row["PREDICTED_REVENUE"] * 0.8
    elif row["DATE"].weekday() == 5 or row["DATE"].weekday() == 6: #Saturday/Sundays
        return row["PREDICTED_REVENUE"] * 1.2
    return row["PREDICTED_REVENUE"]

In [77]:
df_forecast_pandas = df_forecast #.move_to("pandas")

In [78]:
# Adjust for holidays
df_forecast_pandas["PREDICTED_REVENUE"] = df_forecast_pandas.apply(adjust_for_holiday_weekend, axis=1)

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [79]:
df_forecast["PREDICTED_REVENUE"] = df_forecast.apply(adjust_for_holiday_weekend, axis=1)

In [80]:
type(df_forecast)

modin.pandas.dataframe.DataFrame

In [81]:
import altair as alt
alt.data_transformers.disable_max_rows()
df_forecast = df_forecast.move_to("pandas")
chart_predicted = alt.Chart(df_forecast).mark_line(color='blue').encode(
    x='monthdate(DATE):T',
    y='PREDICTED_REVENUE:Q',
    tooltip=['DATE', 'PREDICTED_REVENUE']
).properties(title="Predicted Revenue")
chart_predicted

alt.Chart(...)

In [82]:
df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]
df_transactions_filtered_groupby = df_transactions_filtered.groupby("DATE")["REVENUE"].mean()
# df_transactions_filtered_groupby = df_transactions_filtered_groupby.move_to("pandas")
chart_last_year = alt.Chart(df_transactions_filtered_groupby).mark_line(color='red').encode(
    x='monthdate(DATE):T',
    y='REVENUE:Q',
    tooltip=['DATE', 'REVENUE']
).properties(title="Last Year Revenue")

# Overlay the charts
final_chart = chart_predicted + chart_last_year
final_chart

ValueError: DATE encoding field is specified without a type; the type cannot be automatically inferred because the data is not specified as a pandas.DataFrame.

alt.LayerChart(...)